### pip install & import

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
#import pandas_profiling
import datetime

# 전처리된 파일 불러오기

In [3]:
df = pd.read_csv('../1_preprocessing/preprocessing.csv',encoding='utf-8')
df.head()

,순번,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
0,1,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,"33,121,225,810","4,163,666",사극,15세관람가,일반영화
1,2,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,"31,858,660,536","3,638,287",코미디,15세관람가,일반영화
2,3,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,"27,530,825,087","3,346,172",판타지,12세관람가,일반영화
3,4,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,"25,842,519,330","3,233,946",드라마,12세관람가,일반영화
4,5,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,"33,577,136,860","3,597,963",액션,12세관람가,일반영화


# 관객수 나누기 
#### 기존 회귀 모델 적용. 전국 관객수 만 ~ 십만(1603개) 
#### 분류 모댈 적용. 전국 관객수 80만 이상(744개) 
 10개의 클래스 생성 

# 데이터 리사이징 하기
전국 스크린수, 전국 관객수 좌로 편향된 그래프 pr_report.html(태현폴더) 
<br> 
-> 결국에 데이터 편향이 심해서 회귀모델에서 '전국관객수'가 매우 큰 값에서 예측이 쉽지가 않았음.     
-> 하지만 우리는 어떤 영화에 대해서 "흥행" 하는지를 판단하려면 '전국관객수'가 큰 값에 대한 예측이 필요함.     
-> '전국 관객수' 만 ~ 십만 : 1603개(회귀모델), 80만 이상 : 744개(분류모델)    



#### 따라서 기존의 회귀 모델은 데이터만 정리해서 딥러닝을 돌리면 되고 
#### 분류 모델을 새로 만들어 (클래스 10개) 딥러닝 예측해보기 

### movie_resize2

In [ ]:
# movie_resize = df[(df['전국 관객수'] > 10000)]
# movie_resize.info()

In [4]:
# 전국 스크린수 50 초과 , 전국 관객수 1000명 초과 1100만 미만(1100만 이상은 이상치로 생각) 

movie_resize2 = df[(df['전국 관객수'] > 1000) & (df['전국 스크린수'] > 50)& (df['전국 관객수'] < 11000000)]
movie_resize2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4148 entries, 24 to 7884
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   순번       4148 non-null   int64 
 1   영화명      4148 non-null   object
 2   감독       4148 non-null   object
 3   배급사      4148 non-null   object
 4   개봉일      4148 non-null   object
 5   영화형태     4148 non-null   object
 6   국적       4148 non-null   object
 7   전국 스크린수  4148 non-null   int64 
 8   전국 매출액   4148 non-null   int64 
 9   전국 관객수   4148 non-null   int64 
 10  서울 매출액   4148 non-null   object
 11  서울 관객수   4148 non-null   object
 12  장르       4148 non-null   object
 13  등급       4148 non-null   object
 14  영화구분     4148 non-null   object
dtypes: int64(4), object(11)
memory usage: 518.5+ KB


### 배급사 데이터 변경
KOBIS_개봉일람_2023-03-23 에서 (주)넥스트엔터테인먼트월드(NEW), (주)시너지하우스 (시너지) => 해당 내용을 엑셀에서 (new), (시너지)를 없음으로 변경   
위와 같이 설정을 안하면 위에 데이터가 아예 지워지므로, 엑셀에서 바꾸기를 통해 (new), (시너지) 를 빈칸으로 변경 

In [ ]:
# 점수 처리를 위해 변경

# '( )' 안쪽의 글씨 (ex> (주), (유), (사) etc.,) 함께 삭제
movie_resize2['배급사']=movie_resize2.배급사.str.replace(r"\(.*\)","") 
movie_resize2['배급사']=movie_resize2.배급사.str.replace("㈜","")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("주식회사","")

# (주)씨제이이엔엠 으로 통일
movie_resize2['배급사']=movie_resize2.배급사.str.replace("CJ ENM","씨제이이엔엠")

# 롯데엔터테인먼트 로 통일
movie_resize2['배급사']=movie_resize2.배급사.str.replace("롯데쇼핑롯데엔터테인먼트","롯데엔터테인먼트")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("롯데컬처웍스(주)롯데엔터테인먼트","롯데엔터테인먼트")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("롯데쇼핑(주)롯데시네마","롯데엔터테인먼트")


# 월트디즈니스튜디오스코리아 로 통일
movie_resize2['배급사']=movie_resize2.배급사.str.replace("월트디즈니컴퍼니코리아 유한책임회사","월트디즈니스튜디오스코리아")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("소니픽쳐스릴리징월트디즈니스튜디오스코리아","월트디즈니스튜디오스코리아")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("브에나비스타인터내셔널코리아","월트디즈니스튜디오스코리아")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("소니픽쳐스엔터테인먼트코리아식회사극장배급지점","월트디즈니스튜디오스코리아")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("한국소니픽쳐스릴리징브에나비스타영화㈜","월트디즈니스튜디오스코리아")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("소니픽쳐스릴리징코리아","월트디즈니스튜디오스코리아")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("월트디즈니코리아","월트디즈니스튜디오스코리아")
movie_resize2['배급사']=movie_resize2.배급사.str.replace("월트디즈니컴퍼니코리아","월트디즈니스튜디오스코리아")

# 이십세기폭스코리아 로 통일
movie_resize2['배급사']=movie_resize2.배급사.str.replace("이십세기폭스필름코퍼레이션","이십세기폭스코리아")

# 이름 통일 (KBS 미디어)
movie_resize2['배급사']=movie_resize2.배급사.str.replace("케이비에스미디어","KBS 미디어")

# 이름 통일 (에스케이 텔레콤)
movie_resize2['배급사']=movie_resize2.배급사.str.replace("에스케이플래닛","에스케이텔레콤")

### 국적 데이터 변경

In [ ]:
# 국적 데이터 변경
for idx, row in movie_resize2.iterrows():
    if  row['국적'] not in ['미국', '한국', '일본', '영국', '프랑스','중국','독일']:
        movie_resize2['국적'][idx] = '기타'

In [ ]:
# 값 확인

movie_resize2['국적'].value_counts()


### 날짜 데이터 datetime으로 변경

In [ ]:
# 날짜 데이터를 datetime으로 변경
movie_resize2['개봉일'] = movie_resize2['개봉일'].astype('datetime64')
movie_resize2['연도'] = movie_resize2['개봉일'].dt.year
movie_resize2['월'] = movie_resize2['개봉일'].dt.month

movie_resize2.info()


### 시즌 데이터 변경

In [ ]:
# 시즌 데이터 변경

season = {12 : '겨울',
          1 : '겨울',
          2 : '겨울',
          3 : '봄',
          4 : '봄',
          5 : '봄',
          6 : '여름',
          7 : '여름',
          8 : '여름',
          9 : '가을',
          10 : '가을',
          11 : '가을'}

movie_resize2['계절'] = movie_resize2[['월']].apply(lambda x: x.map(season))


###  총 관객수 추가

In [ ]:
movie_total_people = pd.read_csv('../../data/movie_data/movie_total_people.csv',index_col='연도',encoding='UTF-8')
movie_total_people

In [ ]:
movie_total_people['관객수'] = movie_total_people['관객수'].str.replace(pat=r',', repl=r'', regex=True)
movie_total_people['관객수'] = movie_total_people['관객수'].astype('int64')

In [ ]:
movie_resize2['총 관객수'] = [0 for _ in range(len(movie_resize2))]
movie_resize2.head()

In [ ]:
# 관객수 구하기
for idx, row in movie_total_people.iterrows():
    if idx == 2023 :
        movie_resize2['총 관객수'][movie_resize2['연도'] == idx] = 112805094.0
    else :
        movie_resize2['총 관객수'][movie_resize2['연도'] == idx] = row['관객수']

In [ ]:
# 변경 될때만 따로 풀어서 사용하시길.(파일생성코드)
# movie_resize2.to_csv('./resizing.csv',encoding='utf-8')